In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

from src.models import AttnDecoderRNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0 
EOS_token = 1 

class Lang: 
    def __init__(self, name): 
        self.name = name 
        self.word2idx = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2 

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2idx: 
            self.word2idx[word] = self.n_words 
            self.word2count[word] = 1 
            self.index2word[self.n_words] = word 
            self.n_words += 1 
        else: 
            self.word2count[word] += 1


In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) 
    s = re.sub(r"([.!?])", r" \1", s) 
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading Lines...") 

    lines = open('data/%s-%s.txt'% (lang1, lang2), encoding="utf-8").\
        read().strip().split('\n')

    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    if reverse: 
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else: 
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
Max_length = 10 

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p): 
    return len(p[0].split(' ')) < Max_length and\
        len(p[1].split(' ')) < Max_length and\
        p[1].startswith(eng_prefixes) 

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False): 
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))


Reading Lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['nous sommes deprimes', 'we re depressed']


In [8]:
class EncoderRNN(nn.Module): 
    def __init__(self, input_size, hidden_size, dropout_p=0.1): 
        super(EncoderRNN, self).__init__() 
        self.hidden_size = hidden_size 

        self.embedding = nn.Embedding(input_size, hidden_size) 
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True) 
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input): 
        embedded = self.dropout(self.embedding(input)) 
        output, hidden = self.gru(embedded) 
        return output, hidden

In [9]:
device = next(enc.parameters()).device
print(device)

NameError: name 'enc' is not defined

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(Max_length):
            decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden


In [10]:
enc = EncoderRNN(20, 30)
with torch.no_grad():
    out = enc(torch.randint(high=13,size=(10,24)).to(dtype=torch.long))
    print(out[0].shape)
    print(out[1].shape)


torch.Size([10, 24, 30])
torch.Size([1, 10, 30])


In [11]:
dec = DecoderRNN(30, 25)
with torch.no_grad():
    out_ = dec(out[0], out[1], torch.randint(high=13,size=(10,24)).to(dtype=torch.long))
    print(out_[0].shape)
    print(out_[1].shape)

NameError: name 'DecoderRNN' is not defined

In [12]:
def indexesFromSentence(lang, sentence):
    return [lang.word2idx[word] for word in sentence.split(' ')] + [EOS_token]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, Max_length), dtype=np.int32)
    target_ids = np.zeros((n, Max_length), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        # inp_ids.append(EOS_token)
        # tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [13]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer,
                criterion): 

    total_loss = 0.0 
    for data in dataloader: 
        input_tensor, target_tensor = data 

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_ouputs, encoder_hidden = encoder(input_tensor) 
        decoder_outputs, _, _ = decoder(encoder_ouputs, encoder_hidden, target_tensor) 

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward() 

        encoder_optimizer.step() 
        decoder_optimizer.step() 

        total_loss += loss.item() 
    return total_loss / len(dataloader)

In [14]:
import time 
import math 

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent): 
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
    

In [15]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [16]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [17]:
def evaluate(encoder, decoder, sentence, input_lang:Lang, output_lang:Lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [18]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

Reading Lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
1m 12s (- 18m 11s) (5 6%) 1.7262
2m 24s (- 16m 50s) (10 12%) 0.9719
3m 29s (- 15m 7s) (15 18%) 0.6527
4m 23s (- 13m 11s) (20 25%) 0.4570
5m 23s (- 11m 51s) (25 31%) 0.3284
6m 36s (- 11m 1s) (30 37%) 0.2412
7m 32s (- 9m 41s) (35 43%) 0.1795
8m 38s (- 8m 38s) (40 50%) 0.1379
9m 39s (- 7m 31s) (45 56%) 0.1085
10m 40s (- 6m 24s) (50 62%) 0.0875
11m 48s (- 5m 22s) (55 68%) 0.0724
12m 43s (- 4m 14s) (60 75%) 0.0626
13m 32s (- 3m 7s) (65 81%) 0.0555
14m 23s (- 2m 3s) (70 87%) 0.0484
15m 14s (- 1m 0s) (75 93%) 0.0459
16m 7s (- 0m 0s) (80 100%) 0.0421


In [100]:
encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

> j ai tres sommeil
= i m very sleepy
< i m very sleepy today <EOS>

> je suis surpris
= i m surprised
< i m surprised to do it the remember <EOS>

> vous etes fort grossieres
= you re very rude
< you re impressed aren t you ? <EOS>

> il va vous conduire a l aeroport
= he is going to drive you to the airport
< he is going to drive you to the airport <EOS>

> nous sommes tous occupes
= we re all busy
< we re all busy right now <EOS>

> vous etes tres riches
= you are very rich
< you are not very good looking for you <EOS>

> il est responsable de cet accident
= he is responsible for the accident
< he is responsible for his three s different <EOS>

> vous n etes pas malade
= you re not sick
< you re not very good at you <EOS>

> je suis touriste
= i am a tourist
< i m a little dizzy <EOS>

> c est toi le plus vieux
= you re the oldest
< you re the only one who can help me <EOS>



In [21]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder_attn = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder_attn = AttnDecoderRNN(hidden_size, output_lang.n_words, device=device).to(device)

train(train_dataloader, encoder_attn, decoder_attn, 100, print_every=5, plot_every=5)

encoder_attn.eval()
decoder_attn.eval()
evaluateRandomly(encoder_attn, decoder_attn)

Reading Lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
0m 47s (- 15m 0s) (5 5%) 1.5662
1m 31s (- 13m 42s) (10 10%) 0.7109
2m 17s (- 13m 1s) (15 15%) 0.3684
3m 0s (- 12m 1s) (20 20%) 0.2025
3m 46s (- 11m 19s) (25 25%) 0.1240
4m 28s (- 10m 27s) (30 30%) 0.0860
5m 10s (- 9m 36s) (35 35%) 0.0654
5m 57s (- 8m 56s) (40 40%) 0.0534
6m 44s (- 8m 14s) (45 45%) 0.0457
7m 26s (- 7m 26s) (50 50%) 0.0411
8m 9s (- 6m 40s) (55 55%) 0.0375
8m 52s (- 5m 54s) (60 60%) 0.0349
9m 34s (- 5m 9s) (65 65%) 0.0328
10m 16s (- 4m 24s) (70 70%) 0.0315
10m 58s (- 3m 39s) (75 75%) 0.0300
11m 47s (- 2m 56s) (80 80%) 0.0295
12m 32s (- 2m 12s) (85 85%) 0.0282
13m 17s (- 1m 28s) (90 90%) 0.0281
13m 59s (- 0m 44s) (95 95%) 0.0267
14m 42s (- 0m 0s) (100 100%) 0.0261
> je suis suppose etre celui qui vous aide
= i m supposed to be the one helping you
< i m supposed to be the one helping you <EOS>

> nous sommes tous infectes
= we re all infected
< we r

In [22]:
evaluateRandomly(encoder_attn, decoder_attn)

> c est un bon chanteur
= he is a good singer
< he is a good singer singer <EOS>

> je suis fiere de mon pere
= i m proud of my father
< i m proud of my father s my father <EOS>

> nous nous appretons justement a manger
= we re just getting ready to eat
< we re just getting ready to eat <EOS>

> ils emmenent marie a la salle des urgences
= they re taking mary to the emergency room
< they re taking mary to the emergency room <EOS>

> tu as completement tort
= you are completely wrong
< you are completely wrong <EOS>

> je travaille aussi vite que je peux
= i m working as fast as i can
< i m working as fast as i can <EOS>

> nous faisons encore des emplettes
= we re still shopping around
< we re still shopping around the wrong number <EOS>

> je suis masochiste
= i m a masochist
< i am a masochist <EOS>

> je ne suis vraiment pas tres bonne en francais
= i m really not very good at french
< i m really not very good at french <EOS>

> je ne suis pas actuellement arme
= i m now unarmed
< i

In [23]:
from sacrebleu import corpus_bleu

def evaluateMetrics(encoder, decoder, n=100):
    hypotheses = []
    references = []
    for i in range(n):
        pair = random.choice(pairs)
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        hypotheses.append(output_sentence)
        references.append(pair[1])
    
    bleu = corpus_bleu(hypotheses, references)
    return bleu


In [24]:
bleu_attn = evaluateMetrics(encoder_attn, decoder_attn)

In [30]:
bleu_attn.score

0.6964380312467363

In [26]:
bleu_rnn = evaluateMetrics(encoder, decoder)
bleu_rnn.score

NameError: name 'encoder' is not defined